In [7]:
import os
import datetime as dt
from typing import Union, Dict, TypedDict, Annotated
import pandas as pd
import yfinance as yf
from ta.momentum import RSIIndicator, StochasticOscillator
from ta.trend import MACD
from ta.volume import volume_weighted_average_price
from tabulate import tabulate
import numpy as np

In [8]:
def get_stock_prices(ticker: str) -> Union[Dict, str]:
    """Fetches technical indicators for a given ticker."""
    try:
        # Fetch data from yfinance with more specific parameters
        data = yf.download(
            ticker,
            start=dt.datetime.now() - dt.timedelta(weeks=24 * 3),
            end=dt.datetime.now(),
            interval='1wk',
            progress=False  # Disable download progress bar
        )
        
        # Ensure the data is not empty
        if data.empty:
            return "Error: No data available for the given ticker."

        # Flatten MultiIndex columns
        data.columns = data.columns.get_level_values(0)
        
        # Ensure we have the necessary columns
        required_columns = ['Close', 'High', 'Low', 'Volume']
        for col in required_columns:
            if col not in data.columns:
                return f"Error: Missing required column '{col}' in the data."

        indicators = {}

        # Prepare Close prices series
        close_prices = data['Close']

        # Function to safely calculate indicators
        def safe_indicator_calculation(indicator_func, *args, **kwargs):
            try:
                return indicator_func(*args, **kwargs)
            except Exception as e:
                print(f"Error calculating indicator: {e}")
                return None

        # RSI Calculation
        rsi_indicator_result = safe_indicator_calculation(
            RSIIndicator, close=close_prices, window=14
        )
        if rsi_indicator_result is not None:
            rsi_series = rsi_indicator_result.rsi().dropna().tail(12)
            indicators["RSI"] = {
                rsi_series.index[i].strftime('%Y-%m-%d'): round(float(value), 2) 
                for i, value in enumerate(rsi_series)
            }

        # Stochastic Oscillator
        sto_indicator_result = safe_indicator_calculation(
            StochasticOscillator, 
            high=data['High'], low=data['Low'], close=data['Close'], window=14
        )
        if sto_indicator_result is not None:
            sto_series = sto_indicator_result.stoch().dropna().tail(12)
            indicators["Stochastic_Oscillator"] = {
                sto_series.index[i].strftime('%Y-%m-%d'): round(float(value), 2) 
                for i, value in enumerate(sto_series)
            }

        # MACD Calculation
        macd_indicator_result = safe_indicator_calculation(
            MACD, close=close_prices
        )
        if macd_indicator_result is not None:
            macd_series = macd_indicator_result.macd().dropna().tail(12)
            indicators["MACD"] = {
                macd_series.index[i].strftime('%Y-%m-%d'): round(float(value), 2) 
                for i, value in enumerate(macd_series)
            }

            macd_signal_series = macd_indicator_result.macd_signal().dropna().tail(12)
            indicators["MACD_Signal"] = {
                macd_signal_series.index[i].strftime('%Y-%m-%d'): round(float(value), 2) 
                for i, value in enumerate(macd_signal_series)
            }

        # VWAP Calculation
        try:
            vwap_series = volume_weighted_average_price(
                high=data['High'], low=data['Low'], 
                close=data['Close'], volume=data['Volume']
            ).dropna().tail(12)
            indicators["VWAP"] = {
                vwap_series.index[i].strftime('%Y-%m-%d'): round(float(value), 2) 
                for i, value in enumerate(vwap_series)
            }
        except Exception as e:
            print(f"Error calculating VWAP: {e}")

        return indicators

    except Exception as e:
        import traceback
        print("Full error traceback:")
        traceback.print_exc()
        return f"Error fetching technical indicators: {str(e)}"

In [9]:
def get_financial_metrics(ticker: str) -> Union[Dict, str]:
    """Fetches key financial ratios for a given ticker."""
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        return {
            'PE_Ratio': info.get('forwardPE'),
            'Price_to_Book': info.get('priceToBook'),
            'Debt_to_Equity': info.get('debtToEquity'),
            'Profit_Margins': info.get('profitMargins')
        }
    except Exception as e:
        return f"Error fetching financial metrics: {str(e)}"

In [10]:
def format_stock_analysis(result):
    """Format the stock analysis result in a readable manner."""
    # Prepare output
    output = ["STOCK ANALYSIS REPORT\n"]
    output.append("=" * 50)
    
    # Financial Metrics Section
    output.append("\nFINANCIAL METRICS:")
    output.append("-" * 20)
    fin_metrics = result.get('Financial_Metrics', {})
    fin_table = []
    for metric, value in fin_metrics.items():
        # Format percentage metrics
        if metric == 'Profit_Margins':
            value = f"{value * 100:.2f}%" if value is not None else "N/A"
        elif value is not None:
            value = f"{value:.2f}"
        fin_table.append([metric.replace('_', ' '), value])
    
    # Convert financial metrics to table
    output.append(tabulate(fin_table, headers=['Metric', 'Value'], tablefmt='grid'))
    
    # Technical Indicators Section
    output.append("\nTECHNICAL INDICATORS:")
    output.append("-" * 25)
    
    # Prepare technical indicators
    tech_indicators = result.get('Technical_Indicators', {})
    if isinstance(tech_indicators, str):
        output.append(tech_indicators)
    else:
        for indicator, data in tech_indicators.items():
            output.append(f"\n{indicator}:")
            ind_table = [[date, value] for date, value in data.items()]
            output.append(tabulate(ind_table, headers=['Date', 'Value'], tablefmt='grid'))
    
    return "\n".join(output)

In [11]:
def analyze_stock(ticker: str):
    """
    Analyze a stock and provide only technical indicators and financial metrics.

    Args:
        ticker (str): Stock ticker symbol

    Returns:
        dict or str: Technical indicators and financial metrics or error message
    """
    try:
        # Fetch technical indicators and financial metrics
        technical_indicators = get_stock_prices(ticker)
        financial_metrics = get_financial_metrics(ticker)

        # Combine and return the results
        return {
            'Technical_Indicators': technical_indicators,
            'Financial_Metrics': financial_metrics
        }

    except Exception as e:
        return f"An error occurred during stock analysis: {str(e)}"

In [12]:
# Example call
if __name__ == "__main__":
    # Analyze stock for Tesla (TSLA)
    result = analyze_stock('TSLA')
    
    # Format and print the result
    formatted_result = format_stock_analysis(result)
    print(formatted_result)

STOCK ANALYSIS REPORT


FINANCIAL METRICS:
--------------------
+----------------+---------+
| Metric         | Value   |
+================+=========+
| PE Ratio       | 144.38  |
+----------------+---------+
| Price to Book  | 21.65   |
+----------------+---------+
| Debt to Equity | 18.08   |
+----------------+---------+
| Profit Margins | 13.08%  |
+----------------+---------+

TECHNICAL INDICATORS:
-------------------------

RSI:
+------------+---------+
| Date       |   Value |
+============+=========+
| 2024-09-30 |   61.32 |
+------------+---------+
| 2024-10-07 |   50.53 |
+------------+---------+
| 2024-10-14 |   51.36 |
+------------+---------+
| 2024-10-21 |   62.62 |
+------------+---------+
| 2024-10-28 |   56.73 |
+------------+---------+
| 2024-11-04 |   68.24 |
+------------+---------+
| 2024-11-11 |   68.1  |
+------------+---------+
| 2024-11-18 |   71.91 |
+------------+---------+
| 2024-11-25 |   69.82 |
+------------+---------+
| 2024-12-02 |   74.56 |
+-----------